# 📦 Amazon Logistics – Modelo de Red Multi-Nivel (2 niveles)
### Universidad del SABES · Ingeniería Logística y Cadena de Valor  
### Campus San Felipe · Octubre 2025  

**Autor:** Equipo de Auditoría y Mejora Logística  
**Versión:** v3.3 · _Distribución con Centros Intermedios_

---
## 🔎 Propósito del notebook
Este notebook modela y optimiza una red logística tipo Amazon / comercio electrónico con distribución escalonada:

- **Nivel 1:** Plantas/Fábricas → Centros de Distribución Regionales (CDs)
- **Nivel 2:** Centros de Distribución → Ciudades de Demanda Final

El objetivo es decidir **cuánto mover por cada ruta** y **qué centros activar**, minimizando el costo total de operación y transporte, mientras se cumple la demanda de cada ciudad.

📌 Esta versión está alineada visualmente y estructuralmente con el cuaderno de 1 nivel (`Amazon_Logistics_SABES_SanFelipe_v3.2.ipynb`) para uso académico en clase.

---
### ⚠ Uso académico interno
Material de apoyo para el curso *Logística y Cadena de Valor* del plan de estudios de Ingeniería Logística, Universidad del SABES.  
Se autoriza su ejecución con fines formativos.  
**No se autoriza su redistribución comercial ni su copia parcial o total con fines distintos a docencia.**  

---

## 1. Librerías requeridas
Dependencias utilizadas en este modelo:

- `pulp` → Optimización lineal entera (minimizar costo total)
- `networkx` → Graficar red logística (nodos / flujos)
- `matplotlib` → Visualización final para reporte
- `numpy` → Utilidad numérica

Si estás en Colab y te marca error en `pulp`, ejecuta esta celda primero:

In [ ]:
# 🔧 Instalar dependencias (Colab / entorno limpio)
try:
    import pulp
except ModuleNotFoundError:
    !pip install pulp

try:
    import networkx as nx
except ModuleNotFoundError:
    !pip install networkx


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *  # PULP_CBC_CMD, LpProblem, etc.
import numpy as np


## 2. Definición del escenario logístico
En este caso estudiamos una red con:

### 🏭 Plantas / Fábricas (oferta)
- Capacidad máxima de producción
- Costo de operación por unidad

### 🏬 Centros de Distribución Regionales (CDs)
- Capacidad de manejo
- Costo de manejo por unidad
- Costo fijo por estar activo (activa o apaga CDs)

### 🏙 Ciudades destino (demanda)
- Demanda mínima que debemos surtir

### 🚚 Costos logística
- Costo de transporte Planta → CD
- Costo de transporte CD → Ciudad

_Todos los costos son en pesos MXN por unidad enviada._

In [ ]:
# ================================
# ESCENARIO BASE (2 NIVELES)
# ================================

# Nivel 1: Plantas / Fábricas
PLANTAS = {
    'Planta_Norte': {
        'capacidad': 80000,
        'ubicacion': 'Monterrey',
        'costo_operacion': 50  # costo fijo por unidad producida
    },
    'Planta_Centro': {
        'capacidad': 100000,
        'ubicacion': 'CDMX',
        'costo_operacion': 45
    },
    'Planta_Bajio': {
        'capacidad': 60000,
        'ubicacion': 'Querétaro',
        'costo_operacion': 48
    },
    'Planta_Sur': {
        'capacidad': 50000,
        'ubicacion': 'Puebla',
        'costo_operacion': 52
    },
}

# Nivel 2: Centros de Distribución (CDs)
CENTROS_DISTRIBUCION = {
    'CD_Norte': {
        'capacidad': 55000,
        'ubicacion': 'Monterrey',
        'costo_manejo': 5  # costo por unidad manejada
    },
    'CD_Centro': {
        'capacidad': 75000,
        'ubicacion': 'CDMX',
        'costo_manejo': 4
    },
    'CD_Bajio': {
        'capacidad': 48000,
        'ubicacion': 'Guadalajara',
        'costo_manejo': 5
    },
    'CD_Sur': {
        'capacidad': 38000,
        'ubicacion': 'Veracruz',
        'costo_manejo': 6
    },
}

,
# Nivel 3: Ciudades (Demanda)
CIUDADES_DEMANDA = {
    'Toluca': 27000,
    'León': 21000,
    'San Luis Potosí': 17000,
    'Cancún': 17000,
    'Mérida': 18000,
    'Veracruz': 20000,
}

# Costos transporte Planta → CD
COSTOS_PLANTA_CD = {
    ('Planta_Norte', 'CD_Norte'): 3.0,
    ('Planta_Norte', 'CD_Centro'): 48.0,
    ('Planta_Norte', 'CD_Bajio'): 42.0,
    ('Planta_Norte', 'CD_Sur'): 85.0,

    ('Planta_Centro', 'CD_Norte'): 50.0,
    ('Planta_Centro', 'CD_Centro'): 2.0,
    ('Planta_Centro', 'CD_Bajio'): 38.0,
    ('Planta_Centro', 'CD_Sur'): 55.0,

    ('Planta_Bajio', 'CD_Norte'): 45.0,
    ('Planta_Bajio', 'CD_Centro'): 20.0,
    ('Planta_Bajio', 'CD_Bajio'): 25.0,
    ('Planta_Bajio', 'CD_Sur'): 65.0,

    ('Planta_Sur', 'CD_Norte'): 90.0,
    ('Planta_Sur', 'CD_Centro'): 12.0,
    ('Planta_Sur', 'CD_Bajio'): 75.0,
    ('Planta_Sur', 'CD_Sur'): 35.0,
}

# Costos transporte CD → Ciudad final
COSTOS_CD_CIUDAD = {
    ('CD_Norte', 'Toluca'): 88.0,
    ('CD_Norte', 'León'): 65.0,
    ('CD_Norte', 'San Luis Potosí'): 64.0,
    ('CD_Norte', 'Cancún'): 175.0,
    ('CD_Norte', 'Mérida'): 187.0,
    ('CD_Norte', 'Veracruz'): 116.0,

    ('CD_Centro', 'Toluca'): 7.0,
    ('CD_Centro', 'León'): 42.0,
    ('CD_Centro', 'San Luis Potosí'): 51.0,
    ('CD_Centro', 'Cancún'): 161.0,
    ('CD_Centro', 'Mérida'): 156.0,
    ('CD_Centro', 'Veracruz'): 72.0,

    ('CD_Bajio', 'Toluca'): 60.0,
    ('CD_Bajio', 'León'): 18.0,
    ('CD_Bajio', 'San Luis Potosí'): 32.0,
    ('CD_Bajio', 'Cancún'): 206.0,
    ('CD_Bajio', 'Mérida'): 215.0,
    ('CD_Bajio', 'Veracruz'): 120.0,

    ('CD_Sur', 'Toluca'): 75.0,
    ('CD_Sur', 'León'): 95.0,
    ('CD_Sur', 'San Luis Potosí'): 85.0,
    ('CD_Sur', 'Cancún'): 145.0,
    ('CD_Sur', 'Mérida'): 140.0,
    ('CD_Sur', 'Veracruz'): 15.0,
}

print("✅ Datos cargados: plantas, centros, ciudades y costos.")

## 3. Modelo de optimización
Definición matemática básica:

- Variable `x[p,cd]`: flujo enviado de Planta `p` al Centro `cd`
- Variable `y[cd,c]`: flujo enviado de Centro `cd` a Ciudad `c`
- Variable binaria `z[cd]`: 1 si el Centro de Distribución se activa

**Función objetivo:** minimizar
- costo transporte Planta→CD +
- costo transporte CD→Ciudad +
- costo de manejo en CD +
- costo fijo por activar un CD

**Restricciones clave:**
1. No exceder la capacidad de cada planta.  
2. Lo que entra a cada CD (desde plantas) debe cubrir lo que sale (a ciudades).  
3. Cada CD tiene capacidad máxima.  
4. Las demandas mínimas de cada ciudad se deben cumplir.  
5. Si un CD no está activo (`z=0`), no puede fluir nada hacia/desde él (big-M).

In [ ]:
def resolver_red_multinivel(plantas, cds, ciudades, costos_p_cd, costos_cd_c,
                           incluir_costos_fijos=True, verbose=True):
    """
    Resuelve problema de transporte de 2 niveles con centros intermedios.

    Args:
        plantas (dict): info de plantas con capacidades
        cds (dict): info de centros de distribución
        ciudades (dict): demandas
        costos_p_cd (dict): costos planta→CD
        costos_cd_c (dict): costos CD→ciudad
        incluir_costos_fijos (bool): activar costo fijo de CD
        verbose (bool): imprimir reporte corto al final

    Returns:
        dict: Resultados con status, costo total, flujos y análisis.
    """

    # Crear problema
    prob = LpProblem("Amazon_MultiNivel_Mexico", LpMinimize)

    # ======================
    # VARIABLES DE DECISIÓN
    # ======================

    rutas_p_cd = [(p, cd) for p in plantas.keys() for cd in cds.keys()]
    rutas_cd_c = [(cd, c) for cd in cds.keys() for c in ciudades.keys()]

    x = LpVariable.dicts("flujo_planta_cd", rutas_p_cd, lowBound=0, cat="Integer")
    y = LpVariable.dicts("flujo_cd_ciudad", rutas_cd_c, lowBound=0, cat="Integer")
    z = LpVariable.dicts("cd_activo", cds.keys(), cat="Binary")

    # ======================
    # FUNCIÓN OBJETIVO
    # ======================

    costo_transporte_1 = lpSum([
        costos_p_cd[(p, cd)] * x[(p, cd)] for (p, cd) in rutas_p_cd
    ])

    costo_transporte_2 = lpSum([
        costos_cd_c[(cd, c)] * y[(cd, c)] for (cd, c) in rutas_cd_c
    ])

    costo_manejo_cd = lpSum([
        cds[cd]['costo_manejo'] * lpSum([x[(p, cd)] for p in plantas.keys()])
        for cd in cds.keys()
    ])

    if incluir_costos_fijos:
        COSTO_FIJO_CD = 10000  # MXN por CD activo
        costo_fijo = lpSum([COSTO_FIJO_CD * z[cd] for cd in cds.keys()])
        prob += costo_transporte_1 + costo_transporte_2 + costo_manejo_cd + costo_fijo
    else:
        prob += costo_transporte_1 + costo_transporte_2 + costo_manejo_cd

    # ======================
    # RESTRICCIONES
    # ======================

    # Capacidad de plantas
    for p in plantas.keys():
        prob += lpSum([x[(p, cd)] for cd in cds.keys()]) <= plantas[p]['capacidad'], \
                f"Cap_Planta_{p}"

    # Balance y capacidad en cada CD
    for cd in cds.keys():
        entrada = lpSum([x[(p, cd)] for p in plantas.keys()])
        salida = lpSum([y[(cd, c)] for c in ciudades.keys()])

        prob += salida <= entrada, f"Balance_Entrada_{cd}"
        prob += entrada <= cds[cd]['capacidad'], f"Cap_CD_{cd}"

        # Activación lógica vía Big-M
        M = cds[cd]['capacidad']
        prob += entrada <= M * z[cd], f"Activacion_{cd}"

    # Satisfacción de demanda en cada ciudad
    for c in ciudades.keys():
        prob += lpSum([y[(cd, c)] for cd in cds.keys()]) >= ciudades[c], \
                f"Demanda_{c}"

    # ======================
    # RESOLVER
    # ======================

    status = prob.solve(PULP_CBC_CMD(msg=0))

    if LpStatus[prob.status] != 'Optimal':
        raise ValueError(f"❌ Solución no óptima: {LpStatus[prob.status]}")

    # ======================
    # EXTRAER RESULTADOS
    # ======================

    flujos_p_cd = {
        (p, cd): int(x[(p, cd)].varValue)
        for (p, cd) in rutas_p_cd
        if x[(p, cd)].varValue and x[(p, cd)].varValue > 0.5
    }

    flujos_cd_c = {
        (cd, c): int(y[(cd, c)].varValue)
        for (cd, c) in rutas_cd_c
        if y[(cd, c)].varValue and y[(cd, c)].varValue > 0.5
    }

    cds_activos = [cd for cd in cds.keys() if z[cd].varValue > 0.5]

    utilizacion_plantas = {
        p: sum(flujos_p_cd.get((p, cd), 0) for cd in cds.keys()) / plantas[p]['capacidad']
        for p in plantas.keys()
    }

    utilizacion_cds = {
        cd: sum(flujos_cd_c.get((cd, c), 0) for c in ciudades.keys()) / cds[cd]['capacidad']
        for cd in cds_activos
    }

    resultado = {
        'status': LpStatus[prob.status],
        'costo_total': value(prob.objective),
        'flujos_planta_cd': flujos_p_cd,
        'flujos_cd_ciudad': flujos_cd_c,
        'cds_activos': cds_activos,
        'utilizacion_plantas': utilizacion_plantas,
        'utilizacion_cds': utilizacion_cds,
    }

    if verbose:
        print(f"✅ Óptimo | Costo total: ${resultado['costo_total']:,.2f} MXN")
        print(f"CDs activos: {resultado['cds_activos']}")

    return resultado

print("✅ Función de optimización lista.")

## 4. Visualización de la red logística
Vamos a construir un grafo dirigido con 3 columnas verticales:

- Izquierda: Plantas (cuadrados azules)
- Centro: Centros de Distribución activos (círculos verdes)
- Derecha: Ciudades finales (triángulos rojos)

El grosor de las flechas es proporcional al flujo enviado.  
Se guarda un `.png` para reporte técnico / clase.

In [ ]:
def visualizar_red_multinivel(resultado, plantas, cds, ciudades, filename='red_multinivel.png'):
    """
    Render de la red multi-nivel usando NetworkX.
    Genera y guarda un PNG con la red logística optimizada.
    """

    fig, ax = plt.subplots(figsize=(18, 12))
    G = nx.DiGraph()

    pos = {}

    # --- NODOS: PLANTAS (columna izquierda x=0)
    n_plantas = len(plantas)
    for i, planta in enumerate(plantas.keys()):
        y_pos = 1 - (i + 1) / (n_plantas + 1)
        pos[planta] = (0, y_pos)
        G.add_node(planta, tipo='planta', capacidad=plantas[planta]['capacidad'])

    # --- NODOS: CDs ACTIVOS (columna centro x=0.5)
    cds_activos = resultado['cds_activos']
    n_cds = len(cds_activos)
    for i, cd in enumerate(cds_activos):
        y_pos = 1 - (i + 1) / (n_cds + 1)
        pos[cd] = (0.5, y_pos)
        G.add_node(cd, tipo='cd', capacidad=cds[cd]['capacidad'])

    # --- NODOS: CIUDADES (columna derecha x=1.0)
    n_ciudades = len(ciudades)
    for i, ciudad in enumerate(ciudades.keys()):
        y_pos = 1 - (i + 1) / (n_ciudades + 1)
        pos[ciudad] = (1.0, y_pos)
        G.add_node(ciudad, tipo='ciudad', demanda=ciudades[ciudad])

    # --- ARISTAS con espesores proporcionales al flujo
    max_flujo_1 = max(resultado['flujos_planta_cd'].values()) if resultado['flujos_planta_cd'] else 1
    for (p, cd), flujo in resultado['flujos_planta_cd'].items():
        ancho = 1 + 5 * (flujo / max_flujo_1)
        G.add_edge(p, cd, weight=flujo, nivel=1, ancho=ancho)

    max_flujo_2 = max(resultado['flujos_cd_ciudad'].values()) if resultado['flujos_cd_ciudad'] else 1
    for (cd, c), flujo in resultado['flujos_cd_ciudad'].items():
        ancho = 1 + 5 * (flujo / max_flujo_2)
        G.add_edge(cd, c, weight=flujo, nivel=2, ancho=ancho)

    # --- DIBUJO DE NODOS
    plantas_nodes = [n for n in G.nodes() if G.nodes[n]['tipo'] == 'planta']
    cd_nodes = [n for n in G.nodes() if G.nodes[n]['tipo'] == 'cd']
    ciudad_nodes = [n for n in G.nodes() if G.nodes[n]['tipo'] == 'ciudad']

    nx.draw_networkx_nodes(G, pos, nodelist=plantas_nodes,
                          node_color='#1565c0', node_size=4000,
                          node_shape='s', ax=ax,
                          edgecolors='black', linewidths=2)

    nx.draw_networkx_nodes(G, pos, nodelist=cd_nodes,
                          node_color='#2ecc71', node_size=3500,
                          node_shape='o', ax=ax,
                          edgecolors='black', linewidths=2)

    nx.draw_networkx_nodes(G, pos, nodelist=ciudad_nodes,
                          node_color='#e74c3c', node_size=2500,
                          node_shape='v', ax=ax,
                          edgecolors='black', linewidths=2)

    # --- DIBUJO DE ARCOS
    edges_nivel1 = [(u, v) for u, v, d in G.edges(data=True) if d['nivel'] == 1]
    anchos_1 = [G[u][v]['ancho'] for u, v in edges_nivel1]
    nx.draw_networkx_edges(G, pos, edgelist=edges_nivel1,
                          edge_color='#1565c0', width=anchos_1, alpha=0.7,
                          arrows=True, arrowsize=25, ax=ax,
                          arrowstyle='->', connectionstyle='arc3,rad=0.1')

    edges_nivel2 = [(u, v) for u, v, d in G.edges(data=True) if d['nivel'] == 2]
    anchos_2 = [G[u][v]['ancho'] for u, v in edges_nivel2]
    nx.draw_networkx_edges(G, pos, edgelist=edges_nivel2,
                          edge_color='#2ecc71', width=anchos_2, alpha=0.7,
                          arrows=True, arrowsize=25, ax=ax,
                          arrowstyle='->', connectionstyle='arc3,rad=0.1')

    # --- ETIQUETAS DE NODOS (utilización % en plantas y CDs)
    labels = {}
    for node in G.nodes():
        tipo = G.nodes[node]['tipo']
        if tipo == 'planta':
            util = resultado['utilizacion_plantas'][node]
            labels[node] = f"{node.replace('Planta_', '')}\n{util:.0%}"
        elif tipo == 'cd':
            util = resultado['utilizacion_cds'][node]
            labels[node] = f"{node.replace('CD_', '')}\n{util:.0%}"
        else:
            labels[node] = node

    nx.draw_networkx_labels(G, pos, labels,
                           font_size=11,
                           font_weight='bold', ax=ax)

    # --- ETIQUETAS DE FLUJO EN ARCOS
    edge_labels = {}
    for u, v in G.edges():
        flujo = G[u][v]['weight']
        if flujo > 1000:
            edge_labels[(u, v)] = f"{flujo/1000:.1f}k"
        else:
            edge_labels[(u, v)] = f"{flujo}"

    nx.draw_networkx_edge_labels(G, pos, edge_labels,
                                 font_size=9, ax=ax)

    # --- TÍTULO / LEYENDA
    titulo = (
        f"Red de Distribución Multi-Nivel · SABES San Felipe\n"
        f"Costo Total: ${resultado['costo_total']:,.2f} MXN | "
        f"CDs Activos: {len(resultado['cds_activos'])}/{len(cds)}"
    )
    ax.set_title(titulo,
                 fontsize=16,
                 fontweight='bold',
                 pad=20)

    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D
    legend_elements = [
        Patch(facecolor='#1565c0', edgecolor='black', label='Plantas'),
        Patch(facecolor='#2ecc71', edgecolor='black', label='Centros Dist.'),
        Patch(facecolor='#e74c3c', edgecolor='black', label='Ciudades'),
        Line2D([0], [0], color='#1565c0', lw=3, label='Flujo Planta→CD'),
        Line2D([0], [0], color='#2ecc71', lw=3, label='Flujo CD→Ciudad'),
    ]
    ax.legend(handles=legend_elements,
              loc='upper left', fontsize=11)

    ax.axis('off')
    plt.tight_layout()

    # Guardar imagen para reporte / evidencia
    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"📁 Visualización guardada como: {filename}")

    return fig

print("✅ Función de visualización lista.")

## 5. Reporte resumido tipo auditoría interna
Este bloque imprime:
- Costo total de la red optimizada
- Qué Centros de Distribución quedaron activos y con qué nivel de uso (%)
- Cuánta producción usó cada planta
- Principales flujos críticos

In [ ]:
def generar_reporte(resultado, plantas, cds, ciudades):
    print("\n" + "="*70)
    print("📊 REPORTE DE RESULTADOS - RED MULTI-NIVEL")
    print("="*70)

    print(f"\n💰 COSTO TOTAL: ${resultado['costo_total']:,.2f} MXN")

    print(f"\n🏬 CENTROS DE DISTRIBUCIÓN ACTIVOS ({len(resultado['cds_activos'])}/{len(cds)}):")
    for cd in resultado['cds_activos']:
        util = resultado['utilizacion_cds'][cd]
        print(f"   • {cd}: Utilización {util:.1%}")

    print(f"\n🏭 UTILIZACIÓN DE PLANTAS:")
    for p, util in resultado['utilizacion_plantas'].items():
        produccion = sum(resultado['flujos_planta_cd'].get((p, cd), 0) for cd in cds.keys())
        print(f"   • {p}: {util:.1%} ({produccion:,} / {plantas[p]['capacidad']:,} unidades)")

    print(f"\n🚚 FLUJOS PRINCIPALES (Planta → CD):")
    flujos_ordenados_1 = sorted(resultado['flujos_planta_cd'].items(), key=lambda x: x[1], reverse=True)
    for (p, cd), flujo in flujos_ordenados_1[:5]:
        print(f"   • {p} → {cd}: {flujo:,} unidades")

    print(f"\n🏙️ FLUJOS PRINCIPALES (CD → Ciudad):")
    flujos_ordenados_2 = sorted(resultado['flujos_cd_ciudad'].items(), key=lambda x: x[1], reverse=True)
    for (cd, c), flujo in flujos_ordenados_2[:5]:
        print(f"   • {cd} → {c}: {flujo:,} unidades")

    print("\n" + "="*70)

print("✅ Función de reporte lista.")

## 6. Ejecución completa del modelo (RUN)
Esta celda:
1. Resuelve el modelo de optimización logística.
2. Imprime el reporte de auditoría.
3. Genera y guarda la imagen de red logística `red_multinivel.png`.

👉 Esta es la celda que se corre en clase para evidencia / captura.

In [ ]:
print("\n🚀 INICIANDO ANÁLISIS DE RED MULTI-NIVEL (2 niveles)")

# 1. Optimizar
resultado = resolver_red_multinivel(
    PLANTAS,
    CENTROS_DISTRIBUCION,
    CIUDADES_DEMANDA,
    COSTOS_PLANTA_CD,
    COSTOS_CD_CIUDAD,
    incluir_costos_fijos=True,
    verbose=True
)

# 2. Reporte auditado
generar_reporte(resultado, PLANTAS, CENTROS_DISTRIBUCION, CIUDADES_DEMANDA)

# 3. Visualización tipo red logística
fig = visualizar_red_multinivel(
    resultado,
    PLANTAS,
    CENTROS_DISTRIBUCION,
    CIUDADES_DEMANDA,
    filename='red_multinivel.png'
)

print("\n✅ ANÁLISIS COMPLETADO")
print("📁 Archivos generados:")
print("   • red_multinivel.png  ← Mapa de flujo logístico optimizado")

---
## 7. Notas finales de uso
- Este material se elaboró para fines académicos, simulación logística y análisis de red de distribución multietapa.
- Cualquier similitud con una red de distribución real de una empresa es con fines de estudio.
- Se permite reutilizar el resultado agregado (mapa, costos totales, utilización) en presentaciones de clase.
- **No se autoriza comercialización ni uso industrial sin permiso escrito.**

👩🏽‍🏫 Ingeniería Logística · Universidad del SABES · San Felipe, Gto.

---
**FIN DEL NOTEBOOK**